# ch11. 금융, 경제 데이터 애플리케이션

- 시간상 고정된 위치에 존재하는 데이터를 나타내기 위해 크로스 섹션이라는 용어를 사용한다.
- 특정 날짜의 S&P 500지수의 모든 주식 종목의 종가는 크로스 섹션이다.
- 여러 시간상에서 여러 개의 데이터를 갖는 크로스 섹션 데이터는 패널이라고 한다.
- 주가와 시가 총액을 함께 갖고 있는 데이터는 패널이다.

## 11.1 데이터 준비

### 11.1.1 시계열과 크로스 섹션 정렬

- 금융 데이터를 다룰 때 가장 많은 시간을 들이는 과정은 데이터를 정렬하는 과정
- 서로 관련있는 2개이 시계열 색인이 완전하게 맞아떨어지지 않거나
- 두 DataFrame 객체의 칼럼이나 로우의 라벨이 맞아떨어지지 않는 경우

- pandas는 산술연산을 통해 자동적으로 데이터를 정렬하는 방식을 취하고 있다.

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [2]:
rng = pd.date_range('20180612', periods=4, freq='B')

In [3]:
rng

DatetimeIndex(['2018-06-12', '2018-06-13', '2018-06-14', '2018-06-15'], dtype='datetime64[ns]', freq='B')

In [4]:
prices = DataFrame({'AAPL': [1.1, 1.2, 1.3, 1.4],
                    'JNJ': [2.1, 2.2, 2.3, 2.4],
                    'SPX': [3.1, 3.2, 3.3, 3.4],
                    'XOM': [4.1, 4.2, 4.3, 4.4]}, index=rng)

In [5]:
prices

,AAPL,JNJ,SPX,XOM
2018-06-12,1.1,2.1,3.1,4.1
2018-06-13,1.2,2.2,3.2,4.2
2018-06-14,1.3,2.3,3.3,4.3
2018-06-15,1.4,2.4,3.4,4.4


In [6]:
volume = DataFrame({'AAPL': [10, 11, 12],
                    'JNJ': [20, 21, 22],
                    'XOM': [30, 31, 32]},
                    index=pd.date_range('20180612', periods=3, freq='B'))

In [7]:
volume

,AAPL,JNJ,XOM
2018-06-12,10,20,30
2018-06-13,11,21,31
2018-06-14,12,22,32


- pandas는 데이터 산술연산을 통해 자동적으로 정렬하고 sum 같은 함수 안에서 누락된 데이터를 제외시킨다.

In [8]:
prices * volume

,AAPL,JNJ,SPX,XOM
2018-06-12,11.0,42.0,NaN,123.0
2018-06-13,13.2,46.2,NaN,130.2
2018-06-14,15.6,50.6,NaN,137.6
2018-06-15,NaN,NaN,NaN,NaN


In [9]:
vwap = (prices * volume).sum() / volume.sum()

In [10]:
vwap

AAPL    1.206061
JNJ     2.203175
SPX          NaN
XOM     4.202151
dtype: float64

In [11]:
vwap.dropna()

AAPL    1.206061
JNJ     2.203175
XOM     4.202151
dtype: float64

- align 메서드는 새롭게 색인된 두 객체의 튜플을 반환한다.

In [12]:
prices.align(volume, join='inner')

(            AAPL  JNJ  XOM
 2018-06-12   1.1  2.1  4.1
 2018-06-13   1.2  2.2  4.2
 2018-06-14   1.3  2.3  4.3,             AAPL  JNJ  XOM
 2018-06-12    10   20   30
 2018-06-13    11   21   31
 2018-06-14    12   22   32)

- 다르게 색인되어 있을 수 있는 Series들을  하나의 DataFrame으로 합칠 수도 있다.

In [13]:
s1 = Series(range(3), index=['a', 'b', 'c'])
s2 = Series(range(4), index=['d', 'b', 'c', 'e'])
s3 = Series(range(3), index=['f', 'a', 'c'])

In [14]:
DataFrame({'one': s1, 'two': s2, 'three': s3})

,one,three,two
a,0.0,1.0,NaN
b,1.0,NaN,1.0
c,2.0,2.0,2.0
d,NaN,NaN,0.0
e,NaN,NaN,3.0
f,NaN,0.0,NaN


- 색인을 명시적으로 지정하고 나머지 데이터를 배제하는 것도 가능하다.

In [15]:
DataFrame({'one': s1, 'two': s2, 'three': s3}, index=list('face'))

,one,three,two
f,NaN,0.0,NaN
a,0.0,1.0,NaN
c,2.0,2.0,2.0
e,NaN,NaN,3.0
